# Showcasing Protoplast Checkpointing in Cell-line Classification Model

## 1. Introduction

This notebook showcases the checkpointing feature in PROTOplast, which enables resuming model training even after interruptions or switching to a different dataset. It demonstrates how to save and load training checkpoints, making it easy to continue model development without starting from scratch. This is particularly useful for long training sessions, experimentation with various datasets, or training across multiple sessions or environments.

In [1]:
import anndata
import glob
import numpy as np
import pandas as pd
import os
import pathlib
import protoplast as pt
import ray
import torch

from anndata.experimental import AnnCollection
from protoplast.scrna.anndata.lightning_models import LinearClassifier
from protoplast.scrna.anndata.trainer import RayTrainRunner
from protoplast.scrna.anndata.torch_dataloader import DistributedAnnDataset
from protoplast.scrna.anndata.torch_dataloader import cell_line_metadata_cb, DistributedCellLineAnnDataset

from ray.train import Checkpoint
from ray.train.lightning import RayDDPStrategy

✓ Applied AnnDataFileManager patch
✓ Applied AnnDataFileManager patch


## 2. Dataset pre-processing

We begin by reading the two datasets used to train the cell-line classification model in this notebook. To ensure compatibility, the model requires that both datasets have the same output dimensions

In the following section, we create a unified view by performing an **inner join** on the two datasets based on shared features. During this step, we:

- Identify and record the **number of output classes** (cell-lines),
- Extract the list of **cell-line** of both dataset.

This alignment is essential to ensure the model receives a consistent input/output structure regardless of the dataset source.

In [2]:
DS_PATHS = ["/mnt/hdd2/tan/tahoe100m/plate1_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad",
           "/mnt/hdd2/tan/tahoe100m/plate2_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad"]
adatas = [anndata.io.read_h5ad(p, backed = "r") for p in DS_PATHS]

In [3]:
# Create a view of all dataset
collection = AnnCollection(adatas, join_vars = "inner")

# Record the cell-lines (output classes) in both datasets
cell_lines = collection.obs.cell_line.unique().tolist()
cell_lines_count = collection.obs.cell_line.nunique()

## 3. Configure training step

In [4]:
thread_per_worker = 12
test_size = 0.2 
val_size = 0.0 # if you have only training and test data, just put val_size = 0.0

## 4. Train on `plate1_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab` dataset

In [5]:
plate1_adata = adatas[0]

In [6]:
plate1_adata.obs.head(n = 5)

,sample,gene_count,tscp_count,mread_count,drugname_drugconc,drug,cell_line,sublibrary,BARCODE,pcnt_mito,S_score,G2M_score,phase,pass_filter,cell_name,plate
BARCODE_SUB_LIB_ID,,,,,,,,,,,,,,,,
01_001_025-lib_841,smp_1495,1676,2441,2892,"[('Infigratinib', 0.05, 'uM')]",Infigratinib,CVCL_0131,lib_841,01_001_025,0.025399,-0.066667,-0.095055,G1,full,A-172,plate1
01_001_026-lib_841,smp_1495,1657,2454,2925,"[('Infigratinib', 0.05, 'uM')]",Infigratinib,CVCL_0480,lib_841,01_001_026,0.042787,0.128571,0.650549,G2M,full,PANC-1,plate1
01_001_048-lib_841,smp_1495,1749,2521,2963,"[('Infigratinib', 0.05, 'uM')]",Infigratinib,CVCL_0293,lib_841,01_001_048,0.056724,0.242857,0.308791,G2M,full,HEC-1-A,plate1
01_001_076-lib_841,smp_1495,834,1038,1258,"[('Infigratinib', 0.05, 'uM')]",Infigratinib,CVCL_0397,lib_841,01_001_076,0.066474,0.009524,0.245788,G2M,full,LS 180,plate1
01_001_088-lib_841,smp_1495,1275,1710,2006,"[('Infigratinib', 0.05, 'uM')]",Infigratinib,CVCL_1097,lib_841,01_001_088,0.028655,-0.100000,-0.085348,G1,full,C32,plate1


In [8]:
# Set up training
trainer = RayTrainRunner(
    LinearClassifier,
    DistributedCellLineAnnDataset,
    model_keys = ["num_genes",
                  "num_classes"],
    metadata_cb = cell_line_metadata_cb,
    sparse_keys = "X"
)

2025-09-19 06:41:41,087	INFO worker.py:1951 -- Started a local Ray instance.


(TrainTrainable pid=425332) ✓ Applied AnnDataFileManager patch
(TrainTrainable pid=425332) ✓ Applied AnnDataFileManager patch


(RayTrainWorker pid=425562) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=425332) Started distributed worker processes: 
(TorchTrainer pid=425332) - (node_id=0006e3752903b255c8b7b4e65afdd75a78a7465fb4a83ffa9a32d9c8, ip=192.168.1.226, pid=425562) world_rank=0, local_rank=0, node_rank=0


(RayTrainWorker pid=425562) ✓ Applied AnnDataFileManager patch
(RayTrainWorker pid=425562) ✓ Applied AnnDataFileManager patch
(RayTrainWorker pid=425562) =========Starting the training on 0 with num threads: 12=========


(RayTrainWorker pid=425562) 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
(RayTrainWorker pid=425562) GPU available: True (cuda), used: True
(RayTrainWorker pid=425562) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=425562) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=425562) /mnt/hdd2/nam/miniconda3/envs/test/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.11 /mnt/hdd2/nam/miniconda3/envs/test/lib/python3.1 ...
(RayTrainWorker pid=425562) You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('me

...

(RayTrainWorker pid=425562) /home/nam/protoplast/src/protoplast/scrna/anndata/torch_dataloader.py:107: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
(RayTrainWorker pid=425562)   return torch.sparse_csr_tensor(
(RayTrainWorker pid=425562) /home/nam/protoplast/src/protoplast/scrna/anndata/torch_dataloader.py:107: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
(RayTrainWorker pid=425562)   return torch.sparse_csr_tensor(


Epoch 0: 100%|██████████| 4297/4297 [01:59<00:00, 36.09it/s, v_num=0, train_loss=0.0371]


(RayTrainWorker pid=425562) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/nam/protoplast_results/TorchTrainer_2025-09-19_06-42-09/TorchTrainer_c4109_00000_0_2025-09-19_06-42-09/checkpoint_000000)
(RayTrainWorker pid=425562) `Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 4297/4297 [01:59<00:00, 36.06it/s, v_num=0, train_loss=0.0371]


(RayTrainWorker pid=425562) [rank0]:[W919 06:44:32.967584019 ProcessGroupNCCL.cpp:1538] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


In [ ]:
result = trainer.train([DS_PATHS[0]],
                       batch_size = 1024,
                       test_size = test_size, 
                       val_size = val_size,
                       num_workers = 1,
                       resource_per_worker = {"GPU": 1, "CPU": thread_per_worker})

## 5. Train on `plate2_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab` dataset

We now have a checkpoint saved after training the classification model using the first dataset. We need to pass into `train()` the path to the checkpoint file.

In [10]:
plate2_adata = adatas[1]

In [11]:
plate2_adata.obs.head(n = 5)

,sample,gene_count,tscp_count,mread_count,drugname_drugconc,drug,cell_line,sublibrary,BARCODE,pcnt_mito,S_score,G2M_score,phase,pass_filter,cell_name,plate
BARCODE_SUB_LIB_ID,,,,,,,,,,,,,,,,
01_001_053-lib_1000,smp_1591,2671,5629,6830,"[('Infigratinib', 0.5, 'uM')]",Infigratinib,CVCL_1119,lib_1000,01_001_053,0.016522,-0.265873,-0.313553,G1,full,CFPAC-1,plate2
01_001_082-lib_1000,smp_1591,2148,3173,3826,"[('Infigratinib', 0.5, 'uM')]",Infigratinib,CVCL_0292,lib_1000,01_001_082,0.025843,0.400794,0.520879,G2M,full,HCT15,plate2
01_001_145-lib_1000,smp_1591,683,886,1073,"[('Infigratinib', 0.5, 'uM')]",Infigratinib,CVCL_1098,lib_1000,01_001_145,0.029345,-0.019841,-0.032967,G1,full,HepG2/C3A,plate2
01_001_175-lib_1000,smp_1591,1845,2786,3368,"[('Infigratinib', 0.5, 'uM')]",Infigratinib,CVCL_0131,lib_1000,01_001_175,0.031587,-0.123016,-0.118498,G1,full,A-172,plate2
01_001_181-lib_1000,smp_1591,1228,1849,2226,"[('Infigratinib', 0.5, 'uM')]",Infigratinib,CVCL_0399,lib_1000,01_001_181,0.015143,0.023810,-0.008791,S,full,LoVo,plate2


In [12]:
ray.shutdown()

In [13]:
# Set up training
trainer = RayTrainRunner(
    LinearClassifier,
    DistributedCellLineAnnDataset,
    model_keys = ["num_genes",
                  "num_classes"],
    metadata_cb = cell_line_metadata_cb,
    sparse_keys = "X"
)

2025-09-19 06:48:44,899	INFO worker.py:1951 -- Started a local Ray instance.


(TrainTrainable pid=434446) ✓ Applied AnnDataFileManager patch
(TrainTrainable pid=434446) ✓ Applied AnnDataFileManager patch


(RayTrainWorker pid=434570) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=434446) Started distributed worker processes: 
(TorchTrainer pid=434446) - (node_id=fc843557d0630d6a1c076c5905064061a13e15ca1dae506f17e46d07, ip=192.168.1.226, pid=434570) world_rank=0, local_rank=0, node_rank=0


(RayTrainWorker pid=434570) ✓ Applied AnnDataFileManager patch
(RayTrainWorker pid=434570) ✓ Applied AnnDataFileManager patch
(RayTrainWorker pid=434570) =========Starting the training on 0 with num threads: 12=========


(RayTrainWorker pid=434570) 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
(RayTrainWorker pid=434570) GPU available: True (cuda), used: True
(RayTrainWorker pid=434570) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=434570) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=434570) /mnt/hdd2/nam/miniconda3/envs/test/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.11 /mnt/hdd2/nam/miniconda3/envs/test/lib/python3.1 ...
(RayTrainWorker pid=434570) You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('me

...

(RayTrainWorker pid=434570) /home/nam/protoplast/src/protoplast/scrna/anndata/torch_dataloader.py:107: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
(RayTrainWorker pid=434570)   return torch.sparse_csr_tensor(


Epoch 0: 100%|██████████| 6336/6336 [03:24<00:00, 31.02it/s, v_num=0, train_loss=0.281]


(RayTrainWorker pid=434570) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/nam/protoplast_results/TorchTrainer_2025-09-19_06-49-40/TorchTrainer_d1335_00000_0_2025-09-19_06-49-40/checkpoint_000000)
(RayTrainWorker pid=434570) `Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 6336/6336 [03:24<00:00, 31.00it/s, v_num=0, train_loss=0.281]


(RayTrainWorker pid=434570) [rank0]:[W919 06:53:31.467319817 ProcessGroupNCCL.cpp:1538] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


In [ ]:
ckpt_path = os.path.join(result.checkpoint.path, "checkpoint.ckpt")

result = trainer.train([DS_PATHS[1]],
                       batch_size = 1024,
                       test_size = test_size, 
                       val_size = val_size,
                       num_workers = 1,
                       resource_per_worker = {"GPU": 1, "CPU": thread_per_worker})

### Conclusion

This brings us to the end of the tutorial notebook.

This workflow highlights using checkpointing in **PROTOplast**, enabling efficient model development across diverse datasets.

Feel free to explore and extend this notebook to suit your own data and use cases!